# Metro Boston Housing Analysis: 2013-2020 

Extract Transform Load  

Metro Boston housing prices and days on market for the period 2013-2020. In this notebook, we access the data, prepare it, and export the prepared results for downstream analysis.

Data accessed from Boston Magazine: "Best places to live, annual single family homes" web urls.  

NCH 2021

In [1]:
import os, glob
import pandas as pd

from pandas.plotting import scatter_matrix, radviz
from housinganalysis import pulldata 

## Configure

Specify whether to read the data directly from the web url, or used csv archive in repo.

In [2]:
source_from_web = False #Acquire data from url [true], or use archived csv

## Helpers

Helper function for data preparation. 

In [3]:
def prepare_data(df):
    """ETL: prepare and standardize data"""

    #Drop unnamed column 
    df = df.drop(columns=["Unnamed: 0"])

    #Corner case, 2017 data deviates from naming conventions metric:year 
    if any( df.columns.str.contains(":") ):
        new_names = []
        for item in df.columns:
            value = item.split(":")
            if len(value) == 2:
                value.reverse()
                value = " ".join(value)
            else:
                value = value[0]
            new_names.append( value )
        df.columns = new_names

    #Corner case, 2019-2020 data deviates from prior naming convention 
    if "One-Year % Change in Price" in df.columns:
        df = df.rename(columns={"One-Year Change": "One-Year Days on Market % Change"})

    #Rename columns with standard modifiers
    df = df.rename(columns={
        "Town":                 "City/Town",
        "Unnamed: 0.1":         "City/Town", 
        "Neighborhood / Town":  "City/Town",
        "1-Year Change":        "One-Year Price % Change",
        "One-Year Change":      "One-Year Price % Change",
        "5-Year Change":        "Five-Year Price % Change",
        "Five-Year Change":     "Five-Year Price % Change",
        "10-Year Change":       "Ten-Year Price % Change",
        "Ten-Year Change":      "Ten-Year Price % Change",
        "One-Year Change.1":    "One-Year Days on Market % Change", 
        "One-Year Change.2":    "One-Year Number of Sales % Change", 
        })

    #Replace substrings in cases were column contains a numeric data (e.g. 2013)
    df.columns= df.columns.str.replace('No. of Sales','Number of Sales',regex=True)
    df.columns= df.columns.str.replace('Percent Change in Price','Price % Change',regex=True)
    df.columns= df.columns.str.replace('DOM','Days on Market',regex=True)

    #Reorder price change columns 
    if "One-Year Price Change" in df.columns and "Five-Year Price Change" in df.columns:
        one_year = df.pop( "One-Year Price Change" )
        five_year = df.pop( "Five-Year Price Change" )

        df.insert(df.columns.get_loc("Ten-Year Price Change"), 
            "Five-Year Price Change", five_year)
        df.insert(df.columns.get_loc("Five-Year Price Change"), 
            "One-Year Price Change", one_year)

    #Remove rows with missing values [all]
    df = df[ df.iloc[:,1:].notna().all(axis=1) ]

    #Identify cols with formatted numeric data as strings, and convert to int
    to_convert = []
    for item in df.columns[1:]:
        if df[item].dtype == "object":
                to_convert.append(item)

    # Convert formatted strings to numerics
    for item in to_convert:
        #df[item] = df[item].str.replace(r'\D', '',regex=True).astype(int)
        df[item] = df[item].str.replace("\$|,|%", '',regex=True).astype(float)


    #Reset the index
    df = df.reset_index(drop=True)

    return df

## Get data from web

Data is published on the web each year as an html table. We'll read each of the eight years into a dataframe and write a csv archive in the repo.

In [4]:
#Note each source contains data for the year prior, in some instances, multiple years prior. 
urls = {
    "2013": "https://www.bostonmagazine.com/best-places-to-live-2014-single-family-homes/",
    "2014": "https://www.bostonmagazine.com/best-places-to-live-2015-single-family-homes/",
    "2015": "https://www.bostonmagazine.com/best-places-to-live-2016-single-family-homes/",
    "2016": "https://www.bostonmagazine.com/best-places-to-live-2017-single-family-homes/",
    "2017": "https://www.bostonmagazine.com/property/top-places-to-live-2018-single-family-homes/",
    "2018": "https://www.bostonmagazine.com/property/top-places-to-live-2019-single-family-homes/",
    "2019": "https://www.bostonmagazine.com/property/single-family-home-prices/",
    "2020": "https://www.bostonmagazine.com/property/single-family-home-price-chart-2020/"
    }
#Example url
urls['2013']

'https://www.bostonmagazine.com/best-places-to-live-2014-single-family-homes/'

In [5]:
if source_from_web:

    #get data for a single year
    thisyear = "2013"
    status = pulldata(thisyear, urls[thisyear])

    # get data for all years 
    for year, url in urls.items():
        pulldata( year, url )

## Preview data 

We use the 2013 and 2014 housing data to get a sense of the steps required for data preparation, which we'll need to in order to prototype a helper function. We'll then use this helper function on the full dataset. Note there are a few corner cases not seen in the 2013 and 2014 years that we handle in the function.   

We'll need to:
+ Drop index column   
+ Standardize column names  
+ Reorder columns  
+ Remove missing values 
+ Fix column data types (e.g. numerics) 

And use the following naming conventions...
+ Price ($): "Year + Market Price"
+ Price change (%): "[One|Five|Ten]-Year Price % Change"
+ Days on market (days): "Year + Days on Market"
+ Days on market change (%): "[One]-Year Days on Market % Change" 

In [6]:
df13 = pd.read_csv( "housingData2013.csv" )
df13.head()

,Unnamed: 0,Unnamed: 0.1,2013 Median Price,2012 Median Price,One-Year Change,2008 Median Price,Five-Year Change,2003 Median Price,10-Year Change,2013 Days on Market,2012 Days on Market,One-Year Change.1
0,0,Abington,"$285,000","$267,000",7%,"$290,000",-2%,"$281,250",1%,89,124,28%
1,1,Acton,"$481,500","$480,000",0%,"$503,600",-4%,"$474,500",1%,73,121,-40%
2,2,Amesbury,"$302,500","$251,600",20%,"$265,250",14%,"$315,850",-4%,112,154,-27%
3,3,Andover,"$545,000","$505,000",8%,"$525,000",4%,"$490,000",11%,71,95,-25%
4,4,Arlington,"$550,000","$515,000",7%,"$475,242",16%,"$430,000",28%,27,41,-34%


In [7]:
#Drop the first column, names are messy so we'll use iloc
df13 = df13.iloc[:, 1:] 

#Fix inconsistent names 
df13 = df13.rename(columns={
    "Unnamed: 0.1":         "Town",
    "One-Year Change":      "One-Year Price % Change",
    "Five-Year Change":     "Five-Year Price % Change",
    "10-Year Change":       "Ten-Year Price % Change",
    "One-Year Change.1":    "One-Year Days on Market % Change" 
    })

# reorder columns 
one_year = df13.pop('One-Year Price % Change')
five_year = df13.pop('Five-Year Price % Change')

df13.insert(df13.columns.get_loc('Ten-Year Price % Change'), 
    'Five-Year Price % Change', five_year)
df13.insert(df13.columns.get_loc('Five-Year Price % Change'), 
    'One-Year Price % Change', one_year)

df13.head() 

#Identify cols with formatted numeric data as strings, and convert to int
to_convert = []
for item in df13.columns[1:]:
   if df13[item].dtype == "object":
        to_convert.append(item)

# Convert formatted strings to numerics
for item in to_convert:
    df13[item] = df13[item].str.replace("\$|,|%", '',regex=True).astype(float)

#reset index 
df13 = df13.reset_index(drop=True)
df13.head()

,Town,2013 Median Price,2012 Median Price,2008 Median Price,2003 Median Price,One-Year Price % Change,Five-Year Price % Change,Ten-Year Price % Change,2013 Days on Market,2012 Days on Market,One-Year Days on Market % Change
0,Abington,285000.0,267000.0,290000.0,281250.0,7.0,-2.0,1.0,89,124,28.0
1,Acton,481500.0,480000.0,503600.0,474500.0,0.0,-4.0,1.0,73,121,-40.0
2,Amesbury,302500.0,251600.0,265250.0,315850.0,20.0,14.0,-4.0,112,154,-27.0
3,Andover,545000.0,505000.0,525000.0,490000.0,8.0,4.0,11.0,71,95,-25.0
4,Arlington,550000.0,515000.0,475242.0,430000.0,7.0,16.0,28.0,27,41,-34.0


2014 Housing data 

In [8]:
df14 = pd.read_csv("housingData2014.csv")
df14.head()

,Unnamed: 0,Town,2014 Median Price,2013 Median Price,One-Year Change,Five-Year Change,10-Year Change,2014 Days on Market,2014 Number of Sales
0,0,Abington,"$303,750","$285,000",6.6%,21.5%,-5.7%,82.0,124.0
1,1,Acton,"$525,000","$481,000",9.1%,8.9%,-3.2%,68.0,207.0
2,2,Amesbury,"$295,000","$302,500",-2.5%,1.7%,-10.3%,101.0,153.0
3,3,Andover,"$579,900","$545,000",6.4%,12.9%,8.4%,73.0,349.0
4,4,Arlington,"$612,000","$550,000",11.3%,29.2%,29.1%,23.0,317.0


In [9]:
#Drop first column
df14 = df14.drop(columns=["Unnamed: 0"])
df14.head()

#Fix inconsistent names 
df14 = df14.rename(columns={
    "One-Year Change":      "One-Year Price % Change",
    "Five-Year Change":     "Five-Year Price % Change",
    "10-Year Change":       "Ten-Year Price % Change",
    })

#Remove rows with missing values [all]
df14 = df14[ df14.iloc[:,1:].notna().all(axis=1) ]

#Identify cols with formatted numeric data as strings, and convert to int
to_convert = []
for item in df14.columns[1:]:
   if df14[item].dtype == "object":
        to_convert.append(item)

# Convert formatted strings to numerics
for item in to_convert:
    df14[item] = df14[item].str.replace("\$|,|%", '',regex=True).astype(float)

#reset index 
df14 = df14.reset_index(drop=True)
df14.head()

,Town,2014 Median Price,2013 Median Price,One-Year Price % Change,Five-Year Price % Change,Ten-Year Price % Change,2014 Days on Market,2014 Number of Sales
0,Abington,303750.0,285000.0,6.6,21.5,-5.7,82.0,124.0
1,Acton,525000.0,481000.0,9.1,8.9,-3.2,68.0,207.0
2,Amesbury,295000.0,302500.0,-2.5,1.7,-10.3,101.0,153.0
3,Andover,579900.0,545000.0,6.4,12.9,8.4,73.0,349.0
4,Arlington,612000.0,550000.0,11.3,29.2,29.1,23.0,317.0


## Prepare and write data 

Use `prepare_data` on the full set, and write the data products to disk as housingDataYYYY-prepared.csv.

In [10]:
# Generate a list of data files 
pattern = "*"+('[0-9]'*4)+".csv"
inventory = sorted( glob.glob(pattern) )
inventory

['housingData2013.csv',
 'housingData2014.csv',
 'housingData2015.csv',
 'housingData2016.csv',
 'housingData2017.csv',
 'housingData2018.csv',
 'housingData2019.csv',
 'housingData2020.csv']

In [11]:
for item in inventory:
    df = pd.read_csv(item)

    #Corner case
    if any(df.columns.str.contains("2020")):
        ohNineCols =  ["2009 Median Price", "2009 DOM", "2009 Sales"]
        df = df.drop(columns=ohNineCols)

    df = prepare_data(df)

    fileparts = item.split(".")
    df.to_csv(f"{fileparts[0]}-prepared.{fileparts[1]}", index=False)